下面这一段是在autodl的gpu上进行下载cat-and-dog数据集的代码

In [ ]:
import kagglehub
import shutil
import os
# 指定下载路径


# 下载数据集
path = kagglehub.dataset_download("marquis03/cats-and-dogs")


download_path = "/root/autodl-tmp"
os.makedirs(download_path, exist_ok=True)

# 移动数据集到目标路径
shutil.move(path, download_path)

print(f"Dataset moved to: {download_path}")



In [ ]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from module import ViT

In [ ]:
# Training settings
batch_size = 64
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 42

In [ ]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=2,
    transformer=efficient_transformer,
    channels=3,
).to(device)

## Traning

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler这是个新东西叫做学习率调度器
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
"""
StepLR：这是 PyTorch 中的一种学习率调度器。
通过在每隔一定的训练周期（epoch）后衰减学习率来帮助模型更好地收敛。StepLR 通过“步长衰减”调整学习率。
step_size=1：每过 step_size 个 epoch 后，学习率就会调整一次。在这个例子中，每训练 1 个 epoch 后学习率就会变化一次。
gamma 是学习率衰减的系数，它控制每次调整学习率时衰减的比例。比如，gamma=0.1 表示每 step_size 个 epoch 后，学习率将减少为原来的 10%。

"""

In [ ]:
for epoch in range(epochs):
    epoch_loss = 0#epoch_loss用来记录每个 epoch 的总损失
    epoch_accuracy = 0#epoch_accuracy 用来记录每个 epoch 总准确率

    for data, label in tqdm(train_loader):#tqdm 是一个进度条工具，显示训练进度
        data = data.to(device)
        label = label.to(device)#将数据和标签移动到 GPU

        output = model(data)#将训练数据传入模型，得到模型的输出（预测值）。
        loss = criterion(output, label)#使用预定义的损失函数（criterion）计算模型预测值和真实标签之间的差距。

        """这三板斧看着就觉得好方便啊~~~~"""
        optimizer.zero_grad()#清除优化器之前的梯度信息。每次更新权重之前，都需要清除上一步计算的梯度。
        loss.backward()#计算损失函数的梯度，这个梯度将用于调整模型的权重。
        optimizer.step()#通过优化器更新模型的权重。

        acc = (output.argmax(dim=1) == label).float().mean()#计算当前 batch 的准确率：
                                                            #output.argmax(dim=1)：获取模型输出的最大值的索引，表示预测的类别。
                                                            #== label：比较预测的类别和实际标签是否相同，返回一个布尔值（True 或 False）。
                                                            #.float().mean()：将布尔值转换为浮点数（True = 1，False = 0），然后计算平均值(一个batch中各项的)，即准确率。
        epoch_accuracy += acc / len(train_loader)#将当前 batch 的准确率加入到总准确率中，并且除以 len(train_loader)（总batch数）
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():#验证阶段，禁用梯度计算
        epoch_val_accuracy = 0
        epoch_val_loss = 0#用来记录验证集的准确率和损失。

        for data, label in valid_loader:#应该是验证花时间少点不用tqdm了
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )